In [1]:
import pynq
from pynq.overlays.base import BaseOverlay
from ctypes import *

ol = BaseOverlay("base.bit")

In [2]:
print('test print...')

test print...


In [3]:
%%microblaze ol.PMODA

#include <spi.h>
#include <stdint.h>
#include <pyprintf.h>

#define CS_A_PIN	0
#define CS_M_PIN	6
#define MOSI_PIN	1
#define MISO_PIN	2
#define CLK_PIN	3

#define BUFF_SIZE	7


#define ACC_X_L_ADDR     0x28
#define MAG_X_L_ADDR     0x28
#define GYRO_X_L_ADDR    0x18

#define CTRL_REG6_XL_ADDR    0x20

#define X_DIR_SEL       0x01
#define Y_DIR_SEL       0x02
#define Z_DIR_SEL       0x03

#define READ_CFG       1<<7 //bit 7 high
#define WRITE_CFG      0x7F //bit 7 low
#define MS_CFG         0xC0 //11000000

#define STATE_ON       0x01
#define STATE_OFF      0x00

#define CTRL_REG5_XL   0x1F
#define CTRL_REG6_XL   0x20
#define CTRL_REG4      0x1E

#define CTRL_REG1_M    0x20
#define CTRL_REG2_M    0x21
#define CTRL_REG3_M    0x22
#define CTRL_REG4_M    0x23
#define CTRL_REG5_M    0x24

#define CTRL_REG1_G    0x10
#define CTRL_REG2_G    0x11
#define CTRL_REG3_G    0x12

#define GYRO_WHO_AM_I  0x0F


int SPI_Write(uint8_t addr, uint8_t val, uint8_t CS)
{
    uint8_t tx_buff[BUFF_SIZE] = {0};
    uint8_t rx_buff[BUFF_SIZE] = {0};
    
    tx_buff[0] = addr & WRITE_CFG;
    tx_buff[1] = val;
    
    spi nav = spi_open(CLK_PIN, MISO_PIN, MOSI_PIN, CS);
    
    spi_transfer(nav, (char*)tx_buff, (char*)rx_buff, 2);
    
    spi_close(nav);
    
    return 0;
}

uint8_t SPI_Read(uint8_t addr, uint8_t CS)
{
    uint8_t tx_buff[BUFF_SIZE] = {0};
    uint8_t rx_buff[BUFF_SIZE] = {0};
    
    tx_buff[0] = addr | READ_CFG;   
    
    spi nav = spi_open(CLK_PIN, MISO_PIN, MOSI_PIN, CS);
    
    spi_transfer(nav, (char*)tx_buff, (char*)rx_buff, 2);
    
    spi_close(nav);
    
    return rx_buff[1];
}

int Nav_Acc_Gyro_Pwr(uint8_t state)
{
    uint8_t temp_val = 0x00;
    
    if(state == STATE_ON)
    {  
        //enable acc output
        SPI_Write(CTRL_REG5_XL, 0x38, CS_A_PIN); //00111000
        //set odr rate 10Hz of acc
         SPI_Write(CTRL_REG6_XL, 0x20, CS_A_PIN); //00100000
        //enable gyro output
        SPI_Write(CTRL_REG4, 0x38, CS_A_PIN);//00111000
        //set odr rate 14.9Hz of gyro
        SPI_Write(CTRL_REG1_G, 0x20, CS_A_PIN); //00100000
    }
    else
    {
        pyprintf("--ACC Power OFF--\r\n");
        SPI_Write(CTRL_REG5_XL, 0x00, CS_A_PIN); 
        SPI_Write(CTRL_REG6_XL, 0x00, CS_A_PIN);
        SPI_Write(CTRL_REG4, 0x00, CS_A_PIN);
        SPI_Write(CTRL_REG1_G, 0x00, CS_A_PIN);
    }
    return 0;
}

int Nav_Mag_Pwr(uint8_t state)
{
    if(state == STATE_ON)
    {
        //pyprintf("--MAG Power ON--\r\n");
        //set medium performance mode for x and y and 10Hz ODR for MAG, 
        SPI_Write(CTRL_REG1_M, 0x30, CS_M_PIN);
        //pyprintf("CTRL_REG1_M, %x\r\n", SPI_Read(CTRL_REG1_M, CS_M_PIN));
        
        //set scale to +-4Gauss
        SPI_Write(CTRL_REG2_M, 0x00, CS_M_PIN);
        //pyprintf("CTRL_REG2_M, %x\r\n", SPI_Read(CTRL_REG2_M, CS_M_PIN));
        
        //disable I2C and enable SPI read and write operations, 
        //set the operating mode to continuous conversion
        SPI_Write(CTRL_REG3_M, 0x00, CS_M_PIN);
        //pyprintf("CTRL_REG3_M, %x\r\n", SPI_Read(CTRL_REG3_M, CS_M_PIN));
        
        //set medium performance mode for z axis
        SPI_Write(CTRL_REG4_M, 0x04, CS_M_PIN);
        //pyprintf("CTRL_REG4_M, %x\r\n", SPI_Read(CTRL_REG4_M, CS_M_PIN));
        
        //continuous update of output registers
        SPI_Write(CTRL_REG5_M, 0x00, CS_M_PIN);
        //pyprintf("CTRL_REG5_M, %x\r\n", SPI_Read(CTRL_REG5_M, CS_M_PIN));
    }
    else
    {
        //pyprintf("--MAG Power OFF--\r\n");
        //power down the instrument
        SPI_Write(CTRL_REG3_M, 0x03, CS_M_PIN);
    }
}

int Nav_Gyro_GetData(uint8_t axis)
{
    uint8_t tx_buff[7] = {0};
    uint8_t rx_buff[7] = {0};
    int16_t gyro_x = 0;
    int16_t gyro_y = 0;
    int16_t gyro_z = 0;
    
    
    tx_buff[0] = GYRO_X_L_ADDR | READ_CFG;
    
    spi nav = spi_open(CLK_PIN, MISO_PIN, MOSI_PIN, CS_A_PIN);
    spi_transfer(nav, (char*)tx_buff, (char*)rx_buff, 7);
    spi_close(nav);
    
    gyro_x = rx_buff[2]<<8 | rx_buff[1];
    gyro_y = rx_buff[4]<<8 | rx_buff[3];
    gyro_z = rx_buff[6]<<8 | rx_buff[5];
    
    //pyprintf("Nav_Gyro_GetData: x: %d\r\n", gyro_x);
    //pyprintf("Nav_Gyro_GetData: y: %d\r\n", gyro_y);
    //pyprintf("Nav_Gyro_GetData: z: %d\r\n", gyro_z);
    
    if(axis == X_DIR_SEL)
    {
        return gyro_x;
    }
    else if(axis == Y_DIR_SEL)
    {
        return gyro_y;
    }
    else 
    {
        return gyro_z;
    }
    return 0;
}
    
    

int Nav_Acc_GetData(uint8_t axis)
{
    uint8_t tx_buff[7] = {0};
    uint8_t rx_buff[7] = {0};
    int16_t acc_x = 0;
    int16_t acc_y = 0;
    int16_t acc_z = 0;
    
    tx_buff[0] = ACC_X_L_ADDR | READ_CFG;
    
    spi nav = spi_open(CLK_PIN, MISO_PIN, MOSI_PIN, CS_A_PIN);
    spi_transfer(nav, (char*)tx_buff, (char*)rx_buff, 7);
    spi_close(nav);
    
    acc_x = rx_buff[2]<<8 | rx_buff[1];
    acc_y = rx_buff[4]<<8 | rx_buff[3];
    acc_z = rx_buff[6]<<8 | rx_buff[5];
  
    if(axis == X_DIR_SEL)
    {
        return acc_x;
    }
    else if(axis == Y_DIR_SEL)
    {
        return acc_y;
    }
    else 
    {
        return acc_z;
    }
    return 0;
}

int Nav_Mag_GetData(uint8_t axis)
{
    uint8_t tx_buff[7] = {0};
    uint8_t rx_buff[7] = {0}; 
    int16_t mag_x = 0;
    int16_t mag_y =0;
    int16_t mag_z =0;
    
    tx_buff[0] = MAG_X_L_ADDR | READ_CFG;
    tx_buff[0] = tx_buff[0] | MS_CFG;
    
    spi nav = spi_open(CLK_PIN, MISO_PIN, MOSI_PIN, CS_M_PIN);
    spi_transfer(nav, (char*)tx_buff, (char*)rx_buff, 7);
    spi_close(nav);
    
    mag_x = rx_buff[2]<<8 | rx_buff[1];
    mag_y = rx_buff[4]<<8 | rx_buff[3];
    mag_z = rx_buff[6]<<8 | rx_buff[5];
    
    if(axis == X_DIR_SEL)
    {
        return mag_x;
    }
    else if(axis == Y_DIR_SEL)
    {
        return mag_y;
    }
    else 
    {
        return mag_z;
    }
    return 0;
}
int simple_func()
{
    
    uint8_t tx_buff[7] = {0};
    uint8_t rx_buff[7] = {0};
    
    int16_t mag_x = 0;
    int16_t mag_y =0;
    int16_t mag_z =0;
    
    tx_buff[0] = GYRO_WHO_AM_I | READ_CFG;
    tx_buff[0] = tx_buff[0] | MS_CFG;
    
    spi nav = spi_open(CLK_PIN, MISO_PIN, MOSI_PIN, CS_A_PIN);
    
    spi_transfer(nav, (char*)tx_buff, (char*)rx_buff, 7);
    
    
    
    //out1 = spi_transfer();
}

In [4]:
mag_stop = Nav_Mag_Pwr(0)

acc_gyro_stop = Nav_Acc_Gyro_Pwr(0)

--ACC Power OFF--


In [5]:
acc_gyro_start = Nav_Acc_Gyro_Pwr(1)

mag_start = Nav_Mag_Pwr(1)

In [24]:
#my_file = open("cal_data_acc.txt", "w")
#for i in range(0,1000):
#    acc_get_data_x = Nav_Acc_GetData(1)
#    acc_get_data_y = Nav_Acc_GetData(2)
#    acc_get_data_z = Nav_Acc_GetData(3)
#    my_file.write('ACC \n')
#    my_file.write(str(acc_get_data_x))
#    my_file.write('\n')
#    my_file.write(str(acc_get_data_y))
#    my_file.write('\n')
#    my_file.write(str(acc_get_data_z))
#    my_file.write("\n \n")
#my_file.close()
#my_file = open("cal_data_gyro.txt", "w")
#for i in range(0,1000):
#    acc_gyro_data_x = Nav_Gyro_GetData(1)
#    acc_gyro_data_y = Nav_Gyro_GetData(2)
#    acc_gyro_data_z = Nav_Gyro_GetData(3)
#    my_file.write('GYRO \n')
#    my_file.write(str(acc_gyro_data_x))
#    my_file.write('\n')
#    my_file.write(str(acc_gyro_data_y))
#    my_file.write('\n')
#    my_file.write(str(acc_gyro_data_z))
#    my_file.write('\n \n')
#my_file.close()    
my_file = open("cal_dat_mag.txt", "w")    
for i in range(0,1000):
    mag_data_x = Nav_Mag_GetData(1)
    mag_data_y= Nav_Mag_GetData(2)
    mag_data_z= Nav_Mag_GetData(3)
    my_file.write('MAG \n')
    my_file.write(str(mag_data_x))
    my_file.write('\n')
    my_file.write(str(mag_data_y))
    my_file.write('\n')
    my_file.write(str(mag_data_z))
    my_file.write('\n \n')
my_file.close()

In [38]:
if(( 400 <= mag_z < 30000))
    bool right = TRUE 
if((mag_x < && mag_x >=))
    bool back = TRUE
if((mag_x <  && mag_x >=))
    bool front = TRUE
if(mag_x < )
    bool left = TRUE
if(mag_z >= && mag_z < )
   bool forward = TRUE
if(mag_z < )
   bool backward = TRUE

if(forward && right)
   bool correct_backward = TRUE
   bool correct_left = TRUE
if(backward && right)
   bool correct_forward = TRUE
   bool correct_left = TRUE
if(forward && left)
   bool correct_backward = TRUE
   bool correct_right = TRUE
if(backward && left)
   bool correct_forward = TRUE
   bool correct_right = TRUE


SyntaxError: invalid syntax (<ipython-input-38-b312f8a7b0f1>, line 1)